# Pytorchによるニューラルネットワークの実装
* pytorchでモデルを生成するに，nn.Moduleのサブクラスとして定義する方法とnn.Sequentialを利用する方法がある．
* 結果が同じになる...謎

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd

In [2]:
#np.random.RandomStateクラスは，擬似乱数を生成するクラス
#RandomStateはクラスなので変数に代入して使う
#引数はシード
rng = np.random.RandomState(123)
#rng.uniform(0,10,10)
#rng.randint(10,size = 10)

random_state = 42

##　モデルの構築 

In [3]:
#活性化関数を定義
def relu(x):
    #ReLU関数を定義
    #一つ目の引数が条件，条件に応じて二つ目か三つ目の引数を出力
    x = torch.where(x > 0, x, torch.zeros_like(x))
    return x


def softmax(x):
    #x.max(axis=1, keepdim=True).valuesはxのaxis=1の方向の最大値をとり，次元をそのままにする
    
    x = torch.cat([x.max(axis=1, keepdim=True).values] * x.size()[1], dim=1)
    x_exp = torch.exp(x)
    return x_exp/torch.cat([x_exp.sum(dim=1, keepdim=True)] * x.size()[1],dim=1)

### モデルの構築1
#### nn.Moduleのサブクラスとして定義する方法

In [4]:
in_dim = 2
hid_dim = 3
out_dim = 2

In [5]:
#分かっていない

#ニューラルネットワークの層を定義
class Dense(nn.Module): #nn.Moduleを継承
    def __init__(self,in_dim,out_dim, function=lambda x:x):
        #super()は親クラスを指すためのメソッド
        #super().__init__()は親クラスの__init__()を実行しているだけ
        super().__init__()
        self.W = nn.Parameter(torch.tensor(rng.uniform(
                      low=-np.sqrt(6/in_dim),
                      high=np.sqrt(6/in_dim),
                      size=(in_dim,out_dim)).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]).astype('float32')))
        self.function = function
        
    def forward(self,x):
        return self.function(torch.matmul(x,self.W) + self.b)

    
#ニューラルネットワーク全体を定義   
class MLP(nn.Module):
    def __init__(self,in_dim,hid_dim,out_dim):
        super().__init__()
        self.linear1 = Dense(in_dim,hid_dim)
        self.linear2 = Dense(hid_dim,out_dim)
        
    def forward(self, x):
        x = relu(self.linear1(x))
        x = softmax(self.linear2(x))
        return x
    
mlp = MLP(in_dim,hid_dim,out_dim)
print(mlp)
print(mlp.parameters())

MLP(
  (linear1): Dense()
  (linear2): Dense()
)
<generator object Module.parameters at 0x40ac4402e0>


### モデルの構築2
#### あらかじめ定義されたレイヤーを用いた実装

In [6]:
class MLP(nn.Module):
    def __init__(self,in_dim,hid_dim,out_dim):
        #親クラスの初期化
        super().__init__()
        #Linearクラスのインスタンスをインスタンス変数にそれぞれ代入する
        #層の定義
        self.linear1 = nn.Linear(in_dim,hid_dim)
        self.linear2 = nn.Linear(hid_dim,out_dim)
        
    def forward(self, x):
        #層の接続を定義
        #self.linear1などはインスタンスだが以下のようにメソッドのようにも書ける
        x = relu(self.linear1(x))
        x = softmax(self.linear2(x))
        return x
    
mlp = MLP(in_dim, hid_dim, out_dim)
print(mlp)
print(mlp.parameters())

MLP(
  (linear1): Linear(in_features=2, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=2, bias=True)
)
<generator object Module.parameters at 0x40ac440ac0>


### モデルの構築3
#### nn.Sequentialを利用する方法

mlp = nn.Sequential(nn.Linear(in_dim,hid_dim),
                    nn.ReLU(),
                    nn.Linear(hid_dim,out_dim),
                    nn.Softmax())
print(mlp)
print(mlp.parameters())

## 最適化関数の定義

In [7]:
#optimizerの定義
optimizer = optim.SGD(mlp.parameters(), lr = 0.1)

#勾配のリセット
optimizer.zero_grad()

#パラメータの更新
optimizer.step()

## 擬似の学習データを生成

In [8]:
# XORをMLPで行う
x = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float)
t = torch.tensor([0, 1, 1, 0], dtype=torch.long)

## 学習

In [9]:
#モデルを訓練モードにする
mlp.train()

for i in range(1000):
    #torch.eyeはtorchでの単位行列の出力
    #単位行列からtのインデックスを取り出すことでone-hotエンコーディングできている
    t_hot = torch.eye(2)[t]
    
    #予測値の出力
    y_pred = mlp(x)
    
    #誤差の計算
    loss = -(t_hot*torch.log(y_pred)).sum(axis=1).mean()
    
    #勾配のリセット
    optimizer.zero_grad()
    #逆伝播
    loss.backward()
    
    #パラメータの更新
    optimizer.step()
    
    if i % 100 == 0:
        print(i, loss.item())
        

0 0.6931471824645996
100 0.6931471824645996
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996


### モデルの保存

In [10]:
#モデルのパラメータ情報の取得
state_dict = mlp.state_dict()
print(state_dict)

#モデルの保存
torch.save(state_dict,"./model.pth")

OrderedDict([('linear1.weight', tensor([[-0.2670,  0.1775],
        [-0.0274, -0.1810],
        [-0.4280, -0.0852]])), ('linear1.bias', tensor([ 0.2064,  0.2410, -0.3738])), ('linear2.weight', tensor([[-0.0042,  0.2049,  0.4790],
        [ 0.1647,  0.0988, -0.3728]])), ('linear2.bias', tensor([-0.2165, -0.5295]))])


### モデルの再読み込み

In [12]:
#適当なモデルを再定義
mlp2 = MLP(in_dim, hid_dim, out_dim)

#学習済みパラメータの再読み込み
state_dict = torch.load("./model.pth")

#学習済みのパラメータを再設定
mlp2.load_sate_dict(state_dict)

AttributeError: 'MLP' object has no attribute 'load_sate_dict'